# Smart Travel App
## Notebook 2: Data Read In

**Group 4, Project 4**  
Team: Dominique Villarreal, Enrique Garcia, Jose Santos  
Project Due Date: June 12, 2023


In [2]:
import json
import os
import pandas as pd
import pprint as pp
import random
import requests
import time
import warnings
warnings.simplefilter("ignore")
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [3]:
# Import csv's & load df's
# Denver
#merged_file_path = '/Users/josesantos/LearnPython/BCGit/G4_Project4/Resources/df_Extractions/merged_df.csv'
merged_file_path = '../G4_Project4/Resources/df_Extractions/merged_df.csv'
merged_df = pd.read_csv(merged_file_path)

merged_df.head(2)

FileNotFoundError: [Errno 2] No such file or directory: '../G4_Project4/Resources/df_Extractions/merged_df.csv'

In [3]:
# Process & Organize merged_df
columns_to_delete = ['location', 'coordinates']
merged_df.drop(columns_to_delete, axis=1, inplace=True)

merged_df = merged_df[['city','name', 'rating', 
                       'review_count', 'categories']]

# Access and work with the updated DataFrame
merged_df.head(1)

,city,name,rating,review_count,categories
0,Denver,Jackalope Arts,5.0,8,"['Festivals', 'Arts & Crafts', 'Local Flavor']"


In [4]:
# City - One-hot encoding
#city_encoded_data = pd.get_dummies(merged_df, columns = ['city'])
#city_encoded_data

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19627 entries, 0 to 19626
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   city          19627 non-null  object 
 1   name          19627 non-null  object 
 2   rating        19627 non-null  float64
 3   review_count  19627 non-null  int64  
 4   categories    19627 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 766.8+ KB


In [5]:
# Create encoded df
city_encoded_data = merged_df.copy()
city_encoded_data

,city,name,rating,review_count,categories
0,Denver,Jackalope Arts,5.0,8,"['Festivals', 'Arts & Crafts', 'Local Flavor']"
1,Denver,Special Occasions Events,5.0,4,"['Venues & Event Spaces', 'Party & Event Plann..."
2,Denver,Denver International Wine Festival,4.5,6,['Festivals']
3,Denver,"Denver Mineral, Fossil, Gem & Jewelry Show",4.5,10,['Festivals']
4,Denver,Colorado Festival of Horror,5.0,1,['Festivals']
...,...,...,...,...,...
19622,NewYork,Felice 56,4.0,94,"['Italian', 'Wine Bars']"
19623,NewYork,The Jeffrey Craft Beer & Bites,4.0,728,"['Bars', 'American (New)', 'Coffee & Tea']"
19624,NewYork,Saint Tuesday,4.0,30,"['Lounges', 'Speakeasies', 'Cocktail Bars']"
19625,NewYork,Harlem Nights Bar,4.0,185,"['Bars', 'Music Venues']"


In [6]:
# Explode the 'categories' column
city_encoded_data['categories'] = city_encoded_data['categories'].apply(eval)
city_encoded_data = city_encoded_data.explode('categories')
city_encoded_data

,city,name,rating,review_count,categories
0,Denver,Jackalope Arts,5.0,8,Festivals
0,Denver,Jackalope Arts,5.0,8,Arts & Crafts
0,Denver,Jackalope Arts,5.0,8,Local Flavor
1,Denver,Special Occasions Events,5.0,4,Venues & Event Spaces
1,Denver,Special Occasions Events,5.0,4,Party & Event Planning
...,...,...,...,...,...
19625,NewYork,Harlem Nights Bar,4.0,185,Bars
19625,NewYork,Harlem Nights Bar,4.0,185,Music Venues
19626,NewYork,Cardiff Giant,4.0,56,Bars
19626,NewYork,Cardiff Giant,4.0,56,"Beer, Wine & Spirits"


In [7]:
pathname = '../G4_Project4/Resources/df_Extractions/city_encoded_data.csv'
city_encoded_data.to_csv(pathname, index=False)

In [8]:
# Filter the DataFrame for rows with the city "Denver"
denver_data = city_encoded_data[city_encoded_data['city'] == 'Denver']

# Get the unique categories for Denver
denver_unique_categories = denver_data['categories'].unique()

# Filter the DataFrame for rows with cities other than "Denver"
other_cities_data = city_encoded_data[city_encoded_data['city'] != 'Denver']

# Get the unique categories for the other cities
other_cities_unique_categories = other_cities_data['categories'].unique()

# Get the categories that are unique to Denver
unique_to_denver_categories = set(denver_unique_categories) - set(other_cities_unique_categories)

# Create a DataFrame with the unique categories for Denver
denver_unique_categories_df = pd.DataFrame({'Unique Categories for Denver': list(unique_to_denver_categories)})

# Add city column
denver_unique_categories_df['city'] = 'Denver'

# Rename unique categories column
denver_unique_categories_df['Unique Categories'] = denver_unique_categories_df['Unique Categories for Denver']

# Drop Unique Categories for New York
denver_unique_categories_df.drop('Unique Categories for Denver', axis=1, inplace=True)

# Print the DataFrame
denver_unique_categories_df


,city,Unique Categories
0,Denver,Television Stations
1,Denver,Bingo Halls
2,Denver,Hot Air Balloons
3,Denver,Cannabis Dispensaries
4,Denver,Self Storage
...,...,...
66,Denver,Rugs
67,Denver,Disc Golf
68,Denver,Ski & Snowboard Shops
69,Denver,Fashion


In [9]:
import pandas as pd

# Assuming your DataFrame is named "city_encoded_data"

# Filter the DataFrame for rows with the city "Miami"
miami_data = city_encoded_data[city_encoded_data['city'] == 'Miami']

# Get the unique categories for Miami
miami_unique_categories = miami_data['categories'].unique()

# Filter the DataFrame for rows with cities other than "Miami"
other_cities_data = city_encoded_data[city_encoded_data['city'] != 'Miami']

# Get the unique categories for the other cities
other_cities_unique_categories = other_cities_data['categories'].unique()

# Get the categories that are unique to Miami
unique_to_miami_categories = set(miami_unique_categories) - set(other_cities_unique_categories)

# Create a DataFrame with the unique categories for Miami
miami_unique_categories_df = pd.DataFrame({'Unique Categories for Miami': list(unique_to_miami_categories)})

# Add city column
miami_unique_categories_df['city'] = 'Miami'

# Rename unique categories column
miami_unique_categories_df['Unique Categories'] = miami_unique_categories_df['Unique Categories for Miami']

# Drop Unique Categories for New York
miami_unique_categories_df.drop('Unique Categories for Miami', axis=1, inplace=True)

# Print the DataFrame
miami_unique_categories_df


,city,Unique Categories
0,Miami,Christmas Trees
1,Miami,Landscaping
2,Miami,Wedding Chapels
3,Miami,Resorts
4,Miami,Churches
...,...,...
64,Miami,Eyelash Service
65,Miami,Airsoft
66,Miami,Campgrounds
67,Miami,Speech Therapists


In [10]:
import pandas as pd

# Assuming your DataFrame is named "city_encoded_data"

# Filter the DataFrame for rows with the city "NewYork"
newyork_data = city_encoded_data[city_encoded_data['city'] == 'NewYork']

# Get the unique categories for New York, dropping NaN values
newyork_unique_categories = newyork_data['categories'].dropna().unique()

# Filter the DataFrame for rows with cities other than "NewYork"
other_cities_data = city_encoded_data[city_encoded_data['city'] != 'NewYork']

# Get the unique categories for the other cities, dropping NaN values
other_cities_unique_categories = other_cities_data['categories'].dropna().unique()

# Get the categories that are unique to New York
unique_to_newyork_categories = set(newyork_unique_categories) - set(other_cities_unique_categories)

# Create a DataFrame with the unique categories for New York
newyork_unique_categories_df = pd.DataFrame({'Unique Categories for New York': list(unique_to_newyork_categories)})

# Add city column
newyork_unique_categories_df['city'] = 'New York'

# Rename unique categories column
newyork_unique_categories_df['Unique Categories'] = newyork_unique_categories_df['Unique Categories for New York']

# Drop Unique Categories for New York
newyork_unique_categories_df.drop('Unique Categories for New York', axis=1, inplace=True)

# Print the DataFrame
newyork_unique_categories_df


,city,Unique Categories
0,New York,South African
1,New York,Trivia Hosts
2,New York,Bike Sharing
3,New York,Public Transportation
4,New York,Public Art
5,New York,Fondue
6,New York,Taxis
7,New York,Slovakian
8,New York,Laundromat
9,New York,Ski Resorts


In [11]:
# Concatenate the DataFrames
combined_unique_df = pd.concat([newyork_unique_categories_df, miami_unique_categories_df, denver_unique_categories_df], ignore_index=True)

# Print the combined DataFrame
combined_unique_df

# Export to csv for review
pathname = '../G4_Project4/Resources/df_Extractions/combined_unique_categories.csv'
combined_unique_df.to_csv(pathname, index=False)


In [12]:
# Call in the selected unique df
# Import csv's & load df's
# Denver
selected_unique_categories_file_path = '../G4_Project4/Resources/df_Extractions/combined_unique_categories.csv'
selected_unique_cat_df = pd.read_csv(selected_unique_categories_file_path)

selected_unique_cat_df.head(200)

,city,Unique Categories
0,New York,South African
1,New York,Trivia Hosts
2,New York,Bike Sharing
3,New York,Public Transportation
4,New York,Public Art
...,...,...
176,Denver,Rugs
177,Denver,Disc Golf
178,Denver,Ski & Snowboard Shops
179,Denver,Fashion


In [13]:
# Categories - labelEncoder
# Create a label encoder
label_encoder = LabelEncoder()

# Apply label encoding to the 'categories' column
city_encoded_data['categories_encoded'] = label_encoder.fit_transform(city_encoded_data['categories'])

city_encoded_data

,city,name,rating,review_count,categories,categories_encoded
0,Denver,Jackalope Arts,5.0,8,Festivals,220
0,Denver,Jackalope Arts,5.0,8,Arts & Crafts,38
0,Denver,Jackalope Arts,5.0,8,Local Flavor,348
1,Denver,Special Occasions Events,5.0,4,Venues & Event Spaces,605
1,Denver,Special Occasions Events,5.0,4,Party & Event Planning,416
...,...,...,...,...,...,...
19625,NewYork,Harlem Nights Bar,4.0,185,Bars,58
19625,NewYork,Harlem Nights Bar,4.0,185,Music Venues,384
19626,NewYork,Cardiff Giant,4.0,56,Bars,58
19626,NewYork,Cardiff Giant,4.0,56,"Beer, Wine & Spirits",70


In [14]:
filtered_city_data = city_encoded_data[city_encoded_data['categories'].isin(selected_unique_cat_df['Unique Categories'])]
filtered_city_df = filtered_city_data
filtered_city_df.head(500)

,city,name,rating,review_count,categories,categories_encoded
0,Denver,Jackalope Arts,5.0,8,Local Flavor,348
7,Denver,Cherry Creek Holiday Market,3.5,17,Public Markets,467
18,Denver,Brewability & Pizzability,5.0,99,Special Education,536
20,Denver,The Brew Hut,5.0,121,Brewing Supplies,97
39,Denver,Altitude Brewing & Supply,4.5,16,Brewing Supplies,97
...,...,...,...,...,...,...
19413,NewYork,The Lavaux,4.5,100,Fondue,231
19536,NewYork,Skillman Music Recording Studio,5.0,2,Music Production Services,383
19578,NewYork,Mamak House,4.0,133,Malaysian,354
19592,NewYork,Amor y Amargo,4.0,507,Vermouth Bars,606


In [15]:
# Create the city_category_predict_df
city_category_predict_df = filtered_city_df[['city', 'categories','categories_encoded']]
city_category_predict_df

,city,categories,categories_encoded
0,Denver,Local Flavor,348
7,Denver,Public Markets,467
18,Denver,Special Education,536
20,Denver,Brewing Supplies,97
39,Denver,Brewing Supplies,97
...,...,...,...
19413,NewYork,Fondue,231
19536,NewYork,Music Production Services,383
19578,NewYork,Malaysian,354
19592,NewYork,Vermouth Bars,606


In [16]:
# select X & y and reshape df
X = city_category_predict_df['categories_encoded'].values.reshape(-1, 1)
y = city_category_predict_df['city']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Set to keep track of predicted cities and their activities
predicted_data = []

# Set to keep track of unique experiences
unique_experiences = set()

# Loop until Denver, Miami, and NewYork have been predicted
while len(predicted_data) < 3:
    # Generate a random sample of 10 categories
    sample_categories = random.sample(list(city_category_predict_df['categories_encoded']), 10)
    sample_categories_encoded = pd.DataFrame({'categories_encoded': sample_categories})

    # Predict the city based on the random group of 10 categories
    prediction = model.predict(sample_categories_encoded)
    predicted_city = prediction[0]

    # Check if the predicted city is one of Denver, Miami, or NewYork
    if predicted_city in ['Denver', 'Miami', 'NewYork']:
        # Get the actual category names
        category_names = city_category_predict_df.loc[city_category_predict_df['categories_encoded'].isin(sample_categories), 'categories'].unique()

        # Get the top activities for each sample category
        activities_data = []
        for category in sample_categories:
            # Convert category back to its name
            category_name = city_category_predict_df.loc[city_category_predict_df['categories_encoded'] == category, 'categories'].iloc[0]

            filtered_activities = filtered_city_df[(filtered_city_df['city'] == predicted_city) & (filtered_city_df['categories_encoded'] == category)]
            sorted_activities = filtered_activities.sort_values(['rating', 'review_count'], ascending=[False, False])
            top_activities = sorted_activities[['name', 'rating', 'review_count']].head(3)

            for _, activity in top_activities.iterrows():
                activity_name = activity['name']
                rating = activity['rating']
                review_count = activity['review_count']
                
                # Create a unique experience key
                experience_key = (category_name, predicted_city, activity_name)

                # Check if the experience is unique
                if experience_key not in unique_experiences:
                    # Add the experience to the set of unique experiences
                    unique_experiences.add(experience_key)

                    # Append the predicted activity to the list
                    activities_data.append({
                        'Sample Category': category_name,
                        'Predicted City': predicted_city,
                        'Experience Name': activity_name,
                        'Rating': rating,
                        'Review Count': review_count
                    })
        
        # Append the predicted activities to the list
        predicted_data.extend(activities_data)

# Create a DataFrame from the predicted data
predicted_df = pd.DataFrame(predicted_data)

# Print the final predicted data DataFrame
print("Predicted Data:")
print(predicted_df)


Predicted Data:
  Sample Category Predicted City                      Experience Name  Rating  \
0        Pedicabs        NewYork  Pedicab the Park With Laramie Flick     5.0   
1        Pedicabs        NewYork                       NYC Adventures     5.0   
2        Pedicabs        NewYork                          7evenwheels     5.0   
3      Laundromat        NewYork        Sunshine Laundromat & Pinball     4.5   
4         Piadina        NewYork                       Non Solo Piada     4.5   
5        Scottish        NewYork                        Caledonia Bar     4.5   

   Review Count  
0            16  
1             7  
2             4  
3           120  
4           418  
5           269  


In [17]:
predicted_df

,Sample Category,Predicted City,Experience Name,Rating,Review Count
0,Pedicabs,NewYork,Pedicab the Park With Laramie Flick,5.0,16
1,Pedicabs,NewYork,NYC Adventures,5.0,7
2,Pedicabs,NewYork,7evenwheels,5.0,4
3,Laundromat,NewYork,Sunshine Laundromat & Pinball,4.5,120
4,Piadina,NewYork,Non Solo Piada,4.5,418
5,Scottish,NewYork,Caledonia Bar,4.5,269


In [18]:
# Write the DataFrame to a CSV file
pathname = '../G4_Project4/Resources/Insights/predicted_df.csv'
predicted_df.to_csv(pathname, index=False)

In [19]:
model.score(X_test, y_test)

0.8260869565217391

In [20]:
model.fit(X_test, y_test)

RandomForestClassifier(random_state=42)